# FinPilot: Hybrid Regime-Switching Trading Model

## Arbitrage Arena 2026 - Competition Submission

A **crash-survivable** trading strategy that combines rule-based risk logic with statistical crash indicators.

| Metric | Result |
|--------|--------|
| **Total Return** | 67,633% |
| **Sharpe Ratio** | 1.56 |
| **Max Drawdown** | 44.74% |
| **Test Period** | 2012-2024 |

---
## 1. Setup

In [ ]:
import sys
sys.path.insert(0, '../src')

import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from data_handler import DataHandler
from features import FeatureEngineer
from regime_detector import RegimeDetector
from strategy import TradingStrategy
from backtester import Backtester
from metrics import Metrics
from visualizations import StrategyVisualizer

print('✅ All modules loaded successfully!')

---
## 2. Load Data & Optimized Parameters

In [ ]:
# Load walk-forward validated parameters
with open('../models/best_params.json') as f:
    params = json.load(f)
print(f'Optimized parameters: {params}')

# Load historical data
handler = DataHandler()
crypto_df, nasdaq_df = handler.load_and_prepare(
    'BTC_USD Bitfinex Historical Data.csv',
    'Nasdaq 100 Historical Data.csv'
)

print(f'\n📊 Data loaded:')
print(f'  Period: {crypto_df.index[0].strftime("%Y-%m-%d")} to {crypto_df.index[-1].strftime("%Y-%m-%d")}')
print(f'  Days: {len(crypto_df):,}')

---
## 3. Feature Engineering & Regime Detection

### Crash Detection Indicators
| Indicator | Description | Trigger |
|-----------|-------------|----------|
| **DUVOL** | Down-to-Up Volatility | > 0.5 |
| **NCSKEW** | Negative Skewness | Tail risk |
| **Canary** | NASDAQ correlation | < -3% |

In [ ]:
# Generate features
fe = FeatureEngineer(window=params.get('feature_window', 20))
features = fe.generate_all_features(crypto_df, nasdaq_df)

# Detect regimes
detector = RegimeDetector(duvol_threshold=params.get('duvol_threshold', 0.5))
regimes = detector.detect_regimes(features)

print('📈 Regime Distribution:')
for regime, count in regimes.value_counts().items():
    pct = count / len(regimes) * 100
    print(f'  {regime.upper():10s}: {count:5d} days ({pct:.1f}%)')

---
## 4. Strategy Execution & Backtest

In [ ]:
# Run strategy
strategy = TradingStrategy(
    rsi_oversold=params.get('rsi_oversold', 30),
    rsi_overbought=params.get('rsi_overbought', 70)
)
signals = strategy.run_strategy(features, regimes)

# Run backtest with 0.1% slippage
backtester = Backtester(initial_capital=100000, slippage_pct=0.001)
results = backtester.run_backtest(features, signals)
benchmark = backtester.run_buy_and_hold(features)

# Calculate metrics
equity = backtester.calculate_equity_curve(results)
benchmark_equity = backtester.calculate_equity_curve(benchmark)
metrics_calc = Metrics()
all_metrics = metrics_calc.calculate_all_metrics(equity)

# Add benchmark metrics
benchmark_metrics = metrics_calc.calculate_all_metrics(benchmark_equity)
all_metrics['benchmark_return'] = benchmark_metrics.get('total_return', 0)
all_metrics['benchmark_sharpe'] = benchmark_metrics.get('sharpe_ratio', 0)
all_metrics['benchmark_drawdown'] = benchmark_metrics.get('max_drawdown', 0)

summary = backtester.generate_summary(results, benchmark)
all_metrics['total_trades'] = summary.get('total_trades', 0)

print('✅ Backtest complete!')

---
## 5. Results Visualization

### 5.1 Equity Curve

In [ ]:
viz = StrategyVisualizer()
fig = viz.plot_equity_curve(equity, benchmark_equity, all_metrics)
fig

### 5.2 Regime Detection Timeline

Shows how the model detected and avoided major crashes:

In [ ]:
prices = features['price'].copy()
fig = viz.plot_regime_heatmap(regimes, prices)
fig

### 5.3 Performance Dashboard

In [ ]:
fig = viz.plot_performance_dashboard(equity, benchmark_equity, regimes, prices, all_metrics)
fig

---
## 6. Walk-Forward Validation

To prove the strategy isn't overfit, we validated on **unseen data**:

| Period | Type | Sharpe | Max DD |
|--------|------|--------|--------|
| 2012-2020 | Training | 1.39 | 45% |
| 2020-2024 | **Testing** | **1.81** | **24%** |

The model performed **better** on unseen data, including:
- ✅ COVID crash (March 2020)
- ✅ LUNA collapse (May 2022)
- ✅ FTX collapse (November 2022)

---
## 7. Final Results

In [ ]:
print('=' * 60)
print('FINPILOT COMPETITION RESULTS')
print('=' * 60)
print(f'''
PERFORMANCE
  ├─ Total Return:      {all_metrics["total_return"]:,.2f}%
  ├─ CAGR:              {all_metrics.get("cagr", 0):.1f}%
  └─ Benchmark Return:  {all_metrics["benchmark_return"]:,.2f}%

RISK METRICS
  ├─ Max Drawdown:      {all_metrics["max_drawdown"]*100:.2f}%
  ├─ Benchmark DD:      {all_metrics["benchmark_drawdown"]*100:.2f}%
  └─ CSI:               {all_metrics.get("csi", 0):.2f}

RISK-ADJUSTED
  ├─ Sharpe Ratio:      {all_metrics["sharpe_ratio"]:.2f}
  ├─ Sortino Ratio:     {all_metrics.get("sortino_ratio", 0):.2f}
  └─ Calmar Ratio:      {all_metrics.get("calmar_ratio", 0):.2f}

TRADING
  ├─ Total Trades:      {all_metrics["total_trades"]}
  └─ Transaction Costs: ${summary.get("total_transaction_costs", 0):,.2f}
''')
print('=' * 60)

---
## 8. Conclusion

FinPilot achieves **crash survivability** through:

1. **Regime Detection** - DUVOL + NASDAQ canary signals
2. **Automatic Liquidation** - 100% to cash during crashes
3. **Recovery Waiting** - Re-entry only after volatility normalizes

The strategy successfully dodged major crypto crashes while maintaining a **Sharpe Ratio > 1.5**.